In [1]:
%cd ../..

/home/eli/AnacondaProjects/epych


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import collections
import glob
import functools
import logging
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import quantities as pq

import epych
from epych.statistics import alignment

[striatum:1599283] shmem: mmap: an error occurred while determining whether or not /tmp/ompi.striatum.1000/jf.0/46530560/shared_mem_cuda_pool.striatum could be created.
[striatum:1599283] create_and_attach: unable to create shared memory BTL coordinating structure :: size 134217728 


In [4]:
%matplotlib inline

In [5]:
logging.basicConfig(level=logging.INFO)

In [6]:
CONDITIONS = ["go_gloexp", "go_seqctl", "lo_gloexp", "lonaive", "lo_rndctl", "igo_seqctl"]
PRETRIAL_SECONDS = 0.5
POSTTRIAL_SECONDS = 0.5

In [7]:
NWB_SUBJECTS = glob.glob('/mnt/data/000253/sub-*/')

In [8]:
PILOT_FILES = []

In [9]:
NUM_TRIALS = 0
ODDBALL_ONSET = 0.
ODDBALL_OFFSET = 0.

In [10]:
evoked = {}

In [11]:
aligner = epych.statistics.alignment.AlignmentSummary.unpickle("/mnt/data/000253/hippocampal_alignment", statcls=epych.statistics.alignment.SubcorticalAlignment)

In [12]:
def hippocampal_channels(channels):
    hippocampal_indices = [chan for chan, loc in zip(channels.index.values,
                                                     channels.location.values)
                           if ("DG-" in loc or "CA" in loc)]
    for chan in channels.index.values:
        yield len(hippocampal_indices) > 0 and chan >= hippocampal_indices[0] and chan <= hippocampal_indices[-1]

In [13]:
def hippocampal_align(signal):
    return signal.select_channels(list(hippocampal_channels(signal.channels))).median_filter()

In [14]:
def hippocampal_sampling(sampling):
    signals = {}
    for probe, signal in sampling.signals.items():
        visual_channels = signal.channels.location[["VIS" in loc for loc in signal.channels.location]]
        visual_area = os.path.commonprefix(list(visual_channels.values))
        hippocampal_signal = signal.select_channels(list(hippocampal_channels(signal.channels)))
        if len(hippocampal_signal.channels) > 0:
            signals[visual_area] = hippocampal_signal
    return sampling.__class__(sampling.intervals, sampling.trials, sampling.units, **signals)

In [15]:
def samplings(cond):
    for s, subject_dir in enumerate(sorted(NWB_SUBJECTS)):
        subject = subject_dir.split('/')[-2]
        if not os.path.exists(subject_dir + "/" + cond):
            continue
        sampling = hippocampal_sampling(epych.recording.Sampling.unpickle(subject_dir + cond))
        global ODDBALL_ONSET
        global ODDBALL_OFFSET
        global NUM_TRIALS
        ODDBALL_ONSET += sampling.trials['stim3_start'].sum()
        ODDBALL_OFFSET += sampling.trials['stim3_end'].sum()
        NUM_TRIALS += len(sampling.trials)
        yield sampling
        del sampling
        logging.info("Loaded LFPs for %s in subject %s" % (cond, subject))

In [16]:
erps = {}

In [17]:
def initialize_grandcat(key, signal):
    return epych.statistics.grand.GrandConcatenation(aligner.stats[key])

In [18]:
for cond in CONDITIONS:
    if not os.path.exists("/mnt/data/000253/grandcat_hippocampal_%s" % cond):
        summary = epych.statistic.Summary(epych.statistics.alignment.location_title, initialize_grandcat)
        summary.calculate(samplings(cond))
        cat = summary.results()
        cat.pickle("/mnt/data/000253/grandcat_hippocampal_%s" % cond)
        del summary
        del cat
    logging.info("Grand-concatenated LFPs for condition %s" % cond)

INFO:root:Grand-concatenated LFPs for condition go_gloexp
INFO:root:Grand-concatenated LFPs for condition go_seqctl
INFO:root:Grand-concatenated LFPs for condition lo_gloexp
INFO:root:Loaded LFPs for lonaive in subject sub-621890
INFO:root:Loaded LFPs for lonaive in subject sub-632485
INFO:root:Loaded LFPs for lonaive in subject sub-632487
INFO:root:Loaded LFPs for lonaive in subject sub-637542
INFO:root:Loaded LFPs for lonaive in subject sub-637908
INFO:root:Loaded LFPs for lonaive in subject sub-637909
INFO:root:Loaded LFPs for lonaive in subject sub-640507
INFO:root:Loaded LFPs for lonaive in subject sub-642507
INFO:root:Loaded LFPs for lonaive in subject sub-645322
INFO:root:Loaded LFPs for lonaive in subject sub-645324
INFO:root:Loaded LFPs for lonaive in subject sub-645495
INFO:root:Loaded LFPs for lonaive in subject sub-647836
INFO:root:Loaded LFPs for lonaive in subject sub-649323
INFO:root:Loaded LFPs for lonaive in subject sub-649324
INFO:root:Grand-concatenated LFPs for cond